## Question 1

After successfully completing your degree, hopefully within eight semesters xD, you're embarking on the job search phase. Traditional job hunting can be a bit boring, so why not consider automation as a solution? Luckily, your PAI class introduced you to Selenium, an automation tool.

You've thinking of developing a simple script to navigate rozee.pk and indeed websites, searching for AI/ML Engineer positions. This script extracts crucial information from each job listing, such as job titles, company names, job locations, gender preferences, age requirements, minimum education levels, and experience details (if available). All this data should be neatly organized into a Pandas dataframe, including links to the job listings. Perform analysis on dataframe to find a job which best suits you.

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

# Set up the WebDriver
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

# Navigate to the website
url = 'https://www.rozee.pk/'
driver.get(url)


# Perform the search
search_bar_locator = (By.XPATH, '/html/body/div[2]/div[3]/div[2]/form/div[1]/input[1]')
search_bar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(search_bar_locator))
search_bar.click()
search_bar = driver.find_element(*search_bar_locator)
search_bar.send_keys('AI' + Keys.RETURN)


In [11]:
# Add a delay to wait for the search results to load
time.sleep(10)  # Increase the delay to allow for dynamic loading
move = 0
job_info = []
parent_div = driver.find_element(By.XPATH, '//*[@id="jobs"]')
# job_cards = driver.find_elements(By.XPATH, '/html/body/div[3]/div/div[2]/div[2]/div[3]//div[contains(@class, "job ")]')
job_cards = parent_div.find_elements(By.CLASS_NAME, 'job ')
for job_card in job_cards:
    # Click on the job card to view details
    job_card.click()
    time.sleep(5)
    
    # Extract individual job details from the current job card
    job_Title = driver.find_element(By.CLASS_NAME, 'mrb ').text
    company_name = driver.find_element(By.CLASS_NAME, 'mr5').text
    Job_posted_date = driver.find_element(By.CLASS_NAME, 'col-6').text
    Job_Experience_needed = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[2]/div[3]/div/div[12]/div[2]').text
    Job_skills = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[2]/div[2]/div').text

    # Store job details in a dictionary
    job_info.append({
        'Job Title': job_Title,
        'Company Name': company_name,
        'Job Posted Date': Job_posted_date,
        'Job Experience Needed': Job_Experience_needed,
        'Job Skills': Job_skills
    })
    
    time.sleep(2)
    move = move + 150
    driver.execute_script(f"window.scrollTo(0, {move});")

In [12]:
# Create a DataFrame from the extracted information
df = pd.DataFrame(job_info)

# Print the DataFrame in table format
df

,Job Title,Company Name,Job Posted Date,Job Experience Needed,Job Skills
0,"Manager AI\nMTBC,Multiple Cities, Pakistan","MTBC,Multiple Cities, Pakistan",Industry:,7 Years,Problem Solving\nAnalytical Thinking\nLeadersh...
1,Software Engineer (AI/ML/NLP)\nRiphah Internat...,"Riphah International University,Islamabad, Pak...",Industry:,2 Years - 3 Years,LLM\nAI\nML\nNLP\nSoftware Engineering
2,AI / ML Application Developer\nHRK Software Pa...,"HRK Software Park,Lahore, Pakistan",Industry:,"Nov 21, 2023",Python\nC\n#ML\nDigital Image Processing
3,"AI Engineer\nLegalator AI,Islamabad, Pakistan","Legalator AI,Islamabad, Pakistan",Industry:,"Oct 26, 2023",Flask\nGoogle App Engine\nCloud Systems\nJavaS...
4,"Design Sales Consultant\nAI Technologies,Karac...","AI Technologies,Karachi, Pakistan",Industry:,"Nov 02, 2023",Client Dealing\nCommunication Skills\nTelesales
5,Business Development Officer\nAi Professionals...,"Ai Professionals (Pvt) Limited,Islamabad, Paki...",Industry:,"Nov 13, 2023",Business Development\nSales\nRelationship Buil...
6,"AI Developer\nProduction King,Sialkot, Pakista...","Production King,Sialkot, Pakistan",Industry:,"Nov 03, 2023",Linux\nNLP\nData Science\nJavaScript\nJava\nAr...


## Question 2

Develop a basic movie recommendation system that scrapes data from the IMDb website. The program will retrieve information from the Most Popular Movies page on IMDb. The data to be scraped is

1) Movie Titles <br>
2) Runtime <br>
3) Rating <br>
4) Age Restriction <br>
5) Genre <br>
6) Writer(s) <br>
7) Director(s) <br>
8) Fun Movie Trivia <br> 

The data will be stored in a Pandas dataframe.

Subsequently, the program will prompt the user to specify their preferences for:

1) Genre <br>
2) Minimum Rating<br>
3) Maximum Runtime<br>
4) Age Restriction<br>

It will then filter the dataframe accordingly and select a movie for recommendation. The recommendation will display the movie title, writer, director, and trivia. If there are no movies that match the user's preferences, the program will recommend any movie from the original dataframe.

In [54]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementExceptio

# Set up the WebDriver
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

# Navigate to the website
url = 'https://www.imdb.com/chart/moviemeter'
driver.get(url)

time.sleep(10)

In [88]:
movie_info = []

parent_div = driver.find_element(By.CLASS_NAME, 'ipc-metadata-list--base')

movie_cards = parent_div.find_elements(By.CLASS_NAME, 'cli-post-element')

for movie_card in range(len(movie_cards)):
    # Scroll the movie card into view
    driver.execute_script("arguments[0].scrollIntoView();", movie_cards[movie_card])

    # Wait for the movie card to be clickable
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'cli-post-element')))

    # Click on the movie card to view details
    movie_cards[movie_card].click()
    time.sleep(5)

    try:
        # Extract individual movie details from the current movie card
        movie_Title = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/div[1]/a/h3').text
        movie_Runtime = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/ul[1]/li[2]').text
        movie_Rating = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/div[2]/span').text
        movie_Genre = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/ul[2]').text
        movie_Directors = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[2]/div/div/div[3]/div[1]/ul/li').text
        Age_Restriction = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/ul[1]/li[3]').text
    except NoSuchElementException:
        # Handle the case when age restriction is not provided
        movie_Title = "Not Available"
        movie_Runtime = "Not Available"
        movie_Rating = "Not Available"
        movie_Genre = "Not Available"
        movie_Directors = "Not Available"
        Age_Restriction = "Not Available"
        Age_Restriction = "Not Available"

    # Store movie details in a dictionary
    movie_info.append({
        'movie Title': movie_Title,
        'movie Runtime': movie_Runtime,
        'movie Rating': movie_Rating,
        'movie Genre': movie_Genre,
        'movie Directors': movie_Directors,
        'Age Restriction': Age_Restriction
    })

    time.sleep(2)
    
    # Close the movie details view
    close_button = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div/div[1]/button')
    close_button.click()


In [89]:
# Create a DataFrame from the extracted information
df_movies = pd.DataFrame(movie_info)

# Print the DataFrame in table format
df_movies

,movie Title,movie Runtime,movie Rating,movie Genre,movie Directors,Age Restriction
0,The Killer,1h 58min,6.8\n/10,ActionAdventureCrime,David Fincher,R
1,The Marvels,1h 45min,6.0\n/10,ActionAdventureFantasy,Nia DaCosta,PG-13
2,The Hunger Games: The Ballad of Songbirds & Sn...,2h 37min,7.2\n/10,ActionAdventureDrama,Francis Lawrence,PG-13
3,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
4,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
...,...,...,...,...,...,...
95,Dungeons & Dragons: Honor Among Thieves,2h 14min,7.3\n/10,ActionAdventureComedy,John Francis Daley,PG-13
96,The Breakfast Club,1h 37min,7.8\n/10,ComedyDrama,John Hughes,R
97,The Exorcist: Believer,1h 51min,4.9\n/10,Horror,David Gordon Green,R
98,Gone Girl,2h 29min,8.1\n/10,DramaMysteryThriller,David Fincher,R


In [90]:
# Assuming df is your DataFrame
df_movies = df_movies.replace('\n', '', regex=True)


In [91]:
import re
# Add spaces between genres in 'movie Genre' column
df_movies['movie Genre'] = df_movies['movie Genre'].apply(lambda x: ' '.join(re.findall('[A-Z][^A-Z]*', x)))

# Display the modified DataFrame
df_movies


,movie Title,movie Runtime,movie Rating,movie Genre,movie Directors,Age Restriction
0,The Killer,1h 58min,6.8/10,Action Adventure Crime,David Fincher,R
1,The Marvels,1h 45min,6.0/10,Action Adventure Fantasy,Nia DaCosta,PG-13
2,The Hunger Games: The Ballad of Songbirds & Sn...,2h 37min,7.2/10,Action Adventure Drama,Francis Lawrence,PG-13
3,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
4,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
...,...,...,...,...,...,...
95,Dungeons & Dragons: Honor Among Thieves,2h 14min,7.3/10,Action Adventure Comedy,John Francis Daley,PG-13
96,The Breakfast Club,1h 37min,7.8/10,Comedy Drama,John Hughes,R
97,The Exorcist: Believer,1h 51min,4.9/10,Horror,David Gordon Green,R
98,Gone Girl,2h 29min,8.1/10,Drama Mystery Thriller,David Fincher,R


In [92]:
# Function to convert runtime to minutes
def convert_runtime(runtime):
    if not runtime or 'h' not in runtime or 'min' not in runtime:
        return None
    
    hours = int(re.search(r'\d+', runtime.split('h')[0]).group())
    minutes = int(re.search(r'\d+', runtime.split('min')[0]).group())
    
    return  hours * 60 + minutes


In [93]:
# Apply the function to 'movie Runtime'
df_movies['movie Runtime'] = df_movies['movie Runtime'].apply(convert_runtime)

df_movies

,movie Title,movie Runtime,movie Rating,movie Genre,movie Directors,Age Restriction
0,The Killer,61.0,6.8/10,Action Adventure Crime,David Fincher,R
1,The Marvels,61.0,6.0/10,Action Adventure Fantasy,Nia DaCosta,PG-13
2,The Hunger Games: The Ballad of Songbirds & Sn...,122.0,7.2/10,Action Adventure Drama,Francis Lawrence,PG-13
3,Not Available,NaN,Not Available,Not Available,Not Available,Not Available
4,Not Available,NaN,Not Available,Not Available,Not Available,Not Available
...,...,...,...,...,...,...
95,Dungeons & Dragons: Honor Among Thieves,122.0,7.3/10,Action Adventure Comedy,John Francis Daley,PG-13
96,The Breakfast Club,61.0,7.8/10,Comedy Drama,John Hughes,R
97,The Exorcist: Believer,61.0,4.9/10,Horror,David Gordon Green,R
98,Gone Girl,122.0,8.1/10,Drama Mystery Thriller,David Fincher,R


In [94]:
df_movies['movie Rating'] = df_movies['movie Rating'].str.extract('(\d+\.\d+)').astype(float)

In [95]:
df_movies

,movie Title,movie Runtime,movie Rating,movie Genre,movie Directors,Age Restriction
0,The Killer,61.0,6.8,Action Adventure Crime,David Fincher,R
1,The Marvels,61.0,6.0,Action Adventure Fantasy,Nia DaCosta,PG-13
2,The Hunger Games: The Ballad of Songbirds & Sn...,122.0,7.2,Action Adventure Drama,Francis Lawrence,PG-13
3,Not Available,NaN,NaN,Not Available,Not Available,Not Available
4,Not Available,NaN,NaN,Not Available,Not Available,Not Available
...,...,...,...,...,...,...
95,Dungeons & Dragons: Honor Among Thieves,122.0,7.3,Action Adventure Comedy,John Francis Daley,PG-13
96,The Breakfast Club,61.0,7.8,Comedy Drama,John Hughes,R
97,The Exorcist: Believer,61.0,4.9,Horror,David Gordon Green,R
98,Gone Girl,122.0,8.1,Drama Mystery Thriller,David Fincher,R


In [96]:
import pandas as pd

# Assuming your DataFrame is named df

# Replace 'YourDataFrame' with your actual DataFrame name
df_movies = df_movies.replace('Not Available', pd.NA)
df_movies = df_movies.dropna(how='any')

# Now, df contains rows without "Not Available" values or NaN values in any column
df_movies

,movie Title,movie Runtime,movie Rating,movie Genre,movie Directors,Age Restriction
0,The Killer,61.0,6.8,Action Adventure Crime,David Fincher,R
1,The Marvels,61.0,6.0,Action Adventure Fantasy,Nia DaCosta,PG-13
2,The Hunger Games: The Ballad of Songbirds & Sn...,122.0,7.2,Action Adventure Drama,Francis Lawrence,PG-13
5,Five Nights at Freddy's,61.0,5.5,Horror Mystery Thriller,Emma Tammi,PG-13
7,The Creator,122.0,7.0,Action Adventure Drama,Gareth Edwards,PG-13
...,...,...,...,...,...,...
95,Dungeons & Dragons: Honor Among Thieves,122.0,7.3,Action Adventure Comedy,John Francis Daley,PG-13
96,The Breakfast Club,61.0,7.8,Comedy Drama,John Hughes,R
97,The Exorcist: Believer,61.0,4.9,Horror,David Gordon Green,R
98,Gone Girl,122.0,8.1,Drama Mystery Thriller,David Fincher,R


In [97]:
# Get user preferences
genre_preference = input("Enter your preferred genre: ").lower()
min_rating = float(input("Enter the minimum rating you prefer: "))
max_runtime = int(input("Enter the maximum runtime you prefer (in minutes): "))
age_restriction_preference = input("Enter your preferred age restriction: ").lower()

# Filter DataFrame based on user preferences
filtered_df = df_movies[
    (df_movies['movie Genre'].str.contains(genre_preference, case=False)) &
    (df_movies['movie Rating'] >= min_rating) &
    (df_movies['movie Runtime'].astype(float) <= max_runtime) &
    (df_movies['Age Restriction'].str.contains(age_restriction_preference, case=False))
]

# If no movies match user preferences, recommend any movie from the original DataFrame
if filtered_df.empty:
    print("No movies match your preferences. Recommending any movie from the original DataFrame.")
    recommendation = df_movies.sample(1)
else:
    # Select a movie for recommendation from the filtered DataFrame
    recommendation = filtered_df.sample(1)

# Display the recommended movie
print("\nRecommended Movie:")
recommendation[['movie Title', 'movie Rating', 'movie Genre','movie Runtime', 'movie Directors', 'Age Restriction']]

Enter your preferred genre: crime
Enter the minimum rating you prefer: 7
Enter the maximum runtime you prefer (in minutes): 90
Enter your preferred age restriction: none
No movies match your preferences. Recommending any movie from the original DataFrame.

Recommended Movie:


,movie Title,movie Rating,movie Genre,movie Runtime,movie Directors,Age Restriction
24,Priscilla,7.1,Biography Drama Music,61.0,Sofia Coppola,R


## Question 3

In Question 3, your objective is to find the best available product on Daraz, specifically the Redmi Note 12 smartphone. You will scrape important information from the products listed on the first five pages, which includes the Product Title, Rating, Free Shipping Status, Daraz Mall status, Seller Rating and the Ship on Time score.

To find the best available product, apply the following scoring criteria to each product:

<b>1) </b> The price will be the base score.<br>
<b>2) </b> Award an additional 5,000 points to products with the "Daraz Mall" status.<br>
<b>3) </b> Grant an extra 1,000 points to products with free shipping.<br>
<b>4) </b> Add 10000*((0.4 * (Product Rating / 5)) + (0.4 * (Seller Rating / 100)) + (0.2 * (Shipping Performance / 100)) to the score <br>

In [39]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

# Set up the WebDriver
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

# Navigate to the website
url = 'https://www.daraz.pk/'
driver.get(url)


# Perform the search
search_bar_locator = (By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(search_bar_locator))
search_bar.click()
search_bar = driver.find_element(*search_bar_locator)
search_bar.send_keys('Redmi Note 12' + Keys.RETURN)
time.sleep(10)

In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

product_info = []
time.sleep(10)
# Click on each product card (up to 5 pages)
for page in range(5):
    # Find all product cards on the current page
    product_cards = driver.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

    # Click on each product card on the current page
    for product_card in range(len(product_cards)):
        try:
            # Re-find the product cards inside the loop
            product_cards = driver.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

            product_cards[product_card].click()
            time.sleep(10)

        
            # Extract individual product details from the current product page
            Product_Title = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div').text
            Product_Rating = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]').text
            Free_Shipping_Status = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[1]/div/div/div[3]/div/div[1]/div/div[1]/div[1]/div[1]/span[1]').text
            Daraz_Mall_status = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]/div[2]').text
            Seller_Rating = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]').text
            Ship_Time_score = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]/div[2]').text
            Ship_Time_score = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]/div[2]').text
            reviewer_name = driver.find_element(By.CLASS_NAME, 'user').text
            product_reviews = driver.find_element(By.CLASS_NAME, 'review-content-sl').text
            
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(500, 800);")
            time.sleep(2)
            
            # Store product details in a dictionary
            product_info.append({
                'Product Title': Product_Title,
                'Product Rating': Product_Rating,
                'Free Shipping Status': Free_Shipping_Status,
                'Daraz Mall status': Daraz_Mall_status,
                'Seller Rating': Seller_Rating,
                'Ship Time score': Ship_Time_score,
                'reviewername': reviewer_name,
                'product reviews': product_reviews
            })
            
    
            # Go back to the previous page after extracting reviews
            driver.back()
            
             # Scroll to load more reviews
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        except (NoSuchElementException, StaleElementReferenceException) as e:
            # Handle exceptions and continue to the next iteration
            print(f"Exception: {e}")
            continue

    # Go to the next page if available
    try:
        # Locate the next page button and click it
        next_page_button = driver.find_element(By.CLASS_NAME, 'pagination-next--2pg6Z')
        next_page_button.click()

    
    except NoSuchElementException:
        # Break the loop if the next page button is not found
        break



In [42]:
import pandas as pd

df = pd.DataFrame(product_info)

In [46]:
df

,Product Title,Product Rating,Free Shipping Status,Daraz Mall status,Seller Rating,Ship Time score,reviewername,product reviews
0,Xiaomi Redmi Note 12 || 8GB Ram 128GB Rom || 5...,4.7,Free Delivery,83%,90%,83%,Abdul G.,Assalam u Alaikum Received before due date ori...
1,"Redmi Note 12 - 6.67"" Inch Display - 8GB RAM -...",4.1,Free Delivery,94%,85%,94%,Omair,The product received well packed and within ti...
2,Redmi Note 12 Pro - 8GB Ram 256GB Rom - 5000mA...,4.5,Free Delivery,83%,90%,83%,A***.,#11.11 Phone received on 4th day. Package was ...
3,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,5.0,Free Delivery,100%,93%,100%,Talal F.,waited for Redmi Note 12 Pro's release at Xiao...
4,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,5.0,Free Delivery,97%,95%,97%,Hafiz U.,"Excellent service and product, it was nice exp..."
5,"Redmi Note 12 8GB Ram 128GB Rom, Display Size ...",4.6,Free Delivery,76%,94%,76%,3***3,"Grey order kya thw or blue receive hua, Bx yeh..."
6,Xiaomi Redmi Note 12 - 8GB RAM 128GB/256GB ROM...,4.4,Free Delivery,96%,89%,96%,Waseem S.,Received original product. Delivery time was a...
7,Redmi Note 12 || 8GB Ram 256GB Rom || 5000mAh ...,4.2,Free Delivery,83%,90%,83%,0***0,Excellent product
8,Xiaomi Redmi Note 12 RAM 8 GB ROM 128 Front Ca...,5.0,Free Delivery,100%,93%,100%,Farooq A.,Fast and furious type delivery service. I rece...
9,"Redmi Note 12 Pro - 8GB RAM 256GB ROM - 6.67"" ...",4.0,Free Delivery,100%,88%,100%,Muhammad S.,Good Experience with Daraz so far. it is my th...


In [45]:
df2 = df

In [47]:
df2

,Product Title,Product Rating,Free Shipping Status,Daraz Mall status,Seller Rating,Ship Time score,reviewername,product reviews
0,Xiaomi Redmi Note 12 || 8GB Ram 128GB Rom || 5...,4.7,Free Delivery,83%,90%,83%,Abdul G.,Assalam u Alaikum Received before due date ori...
1,"Redmi Note 12 - 6.67"" Inch Display - 8GB RAM -...",4.1,Free Delivery,94%,85%,94%,Omair,The product received well packed and within ti...
2,Redmi Note 12 Pro - 8GB Ram 256GB Rom - 5000mA...,4.5,Free Delivery,83%,90%,83%,A***.,#11.11 Phone received on 4th day. Package was ...
3,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,5.0,Free Delivery,100%,93%,100%,Talal F.,waited for Redmi Note 12 Pro's release at Xiao...
4,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,5.0,Free Delivery,97%,95%,97%,Hafiz U.,"Excellent service and product, it was nice exp..."
5,"Redmi Note 12 8GB Ram 128GB Rom, Display Size ...",4.6,Free Delivery,76%,94%,76%,3***3,"Grey order kya thw or blue receive hua, Bx yeh..."
6,Xiaomi Redmi Note 12 - 8GB RAM 128GB/256GB ROM...,4.4,Free Delivery,96%,89%,96%,Waseem S.,Received original product. Delivery time was a...
7,Redmi Note 12 || 8GB Ram 256GB Rom || 5000mAh ...,4.2,Free Delivery,83%,90%,83%,0***0,Excellent product
8,Xiaomi Redmi Note 12 RAM 8 GB ROM 128 Front Ca...,5.0,Free Delivery,100%,93%,100%,Farooq A.,Fast and furious type delivery service. I rece...
9,"Redmi Note 12 Pro - 8GB RAM 256GB ROM - 6.67"" ...",4.0,Free Delivery,100%,88%,100%,Muhammad S.,Good Experience with Daraz so far. it is my th...


# Question 4

Having successfully identified the best product, proceed to its product page on Daraz. Your task is to scrape the product's reviews and perform sentiment analysis on these reviews. To conduct sentiment analysis, you can utilize the NLTK (Natural Language Toolkit) library, which will help you determine the sentiment expressed in the text of the reviews.

In [50]:
analysis_df = df2[['reviewername', 'product reviews']].copy()

In [51]:
analysis_df

,reviewername,product reviews
0,Abdul G.,Assalam u Alaikum Received before due date ori...
1,Omair,The product received well packed and within ti...
2,A***.,#11.11 Phone received on 4th day. Package was ...
3,Talal F.,waited for Redmi Note 12 Pro's release at Xiao...
4,Hafiz U.,"Excellent service and product, it was nice exp..."
5,3***3,"Grey order kya thw or blue receive hua, Bx yeh..."
6,Waseem S.,Received original product. Delivery time was a...
7,0***0,Excellent product
8,Farooq A.,Fast and furious type delivery service. I rece...
9,Muhammad S.,Good Experience with Daraz so far. it is my th...


In [53]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Assuming 'product reviews' is the column containing the reviews
reviews = analysis_df['product reviews'].tolist()

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis on each review
sentiments = []

for review in reviews:
    sentiment_score = sia.polarity_scores(review)
    
    # Classify the sentiment based on the compound score
    if sentiment_score['compound'] >= 0.05:
        sentiment = 'Positive'
    elif sentiment_score['compound'] <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'
    
    sentiments.append(sentiment)

# Add the sentiments to the DataFrame
analysis_df['Sentiment'] = sentiments

# Display the DataFrame with sentiments

analysis_df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,reviewername,product reviews,Sentiment
0,Abdul G.,Assalam u Alaikum Received before due date ori...,Positive
1,Omair,The product received well packed and within ti...,Positive
2,A***.,#11.11 Phone received on 4th day. Package was ...,Positive
3,Talal F.,waited for Redmi Note 12 Pro's release at Xiao...,Positive
4,Hafiz U.,"Excellent service and product, it was nice exp...",Positive
5,3***3,"Grey order kya thw or blue receive hua, Bx yeh...",Positive
6,Waseem S.,Received original product. Delivery time was a...,Positive
7,0***0,Excellent product,Positive
8,Farooq A.,Fast and furious type delivery service. I rece...,Negative
9,Muhammad S.,Good Experience with Daraz so far. it is my th...,Positive
